In [1]:
import os, glob
import numpy as np
import pandas as pd
import psycopg2
import sql_queries

In [2]:
conn = psycopg2.connect(
    "host=127.0.0.1 dbname=mydb user=postgres password={}".format(open('password.txt','r').read())
    )
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, '*json'))
        for f in files:
            if 'checkpoint' in os.path.abspath(f):
                break
            else:
                all_files.append(os.path.abspath(f).replace('\\', '/'))

    return all_files

In [4]:
def nan_to_null(f, 
_NULL=psycopg2.extensions.AsIs('NULL'), 
_Float=psycopg2.extensions.Float):

    if not np.isnan(f):
        return _Float(f)
    return _NULL

psycopg2.extensions.register_adapter(float, nan_to_null)

In [5]:
song_files = get_files('data/song_data')

df_song = pd.read_json(song_files[1], lines=True)
df_song = df_song.reindex(sorted(df_song.columns), axis=1).replace({np.nan, None})
df_song.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969


## artists Table
Extract Data for Artists Table
* Select columns for artist ID, name, location, latitude, and longitude
* Use df.values to select just the values from the dataframe
* Index to select the first (only) record in the dataframe
* Convert the array to a list and set it to artist_data

In [6]:
data_artist = df_song[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values[0].tolist()
df_artist = df_song[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]]
print(data_artist,'\n',sql_queries.insert_record.artist)

['ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892] 
 
            INSERT INTO artists
            (artist_id, name, location, latitude, longitude)
            VALUES (%s, %s, %s, %s, %s)
            ON CONFLICT (artist_id) DO NOTHING;
            



### Insert Record into Artist Table

Implement the artist_table_insert query in sql_queries.py and run the cell below to insert a record for this song's artist into the artists table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the artists table in the sparkify database.

In [7]:
cur.execute(sql_queries.insert_record.artist, data_artist)
conn.commit()

## songs Table

Extract Data for Songs Table
* Select columns for song ID, title, artist ID, year, and duration
* Use df.values to select just the values from the dataframe
* Index to select the first (only) record in the dataframe
* Convert the array to a list and set it to song_data


In [8]:
artist_id, artist_latitude, artist_location, artist_longitude, artist_name, duration, num_songs, song_id, title, year = df_song.reset_index(drop=True).values[0]

data_song = [song_id, title, artist_id, year, duration]

### Insert Record into Song Table
Implement the song_table_insert query in sql_queries.py and run the cell below to insert a record for this song into the songs table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the songs table in the sparkify database.

In [9]:
print(data_song,'\n',sql_queries.insert_record.song)

['SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546] 
 
            INSERT INTO songs
            (song_id, title, artist_id, year, duration)
            VALUES (%s, %s, %s, %s, %s)
            ON CONFLICT (song_id) DO NOTHING;
            


In [10]:
cur.execute(sql_queries.insert_record.song, data_song)
conn.commit()

## Process log_data
In this part, you'll perform ETL on the second dataset, log_data, to create the time and users dimensional tables, as well as the songplays fact table.

Let's perform ETL on a single log file and load a single record into each table.

* Use the get_files function provided above to get a list of all log JSON files in data/log_data
* Select the first log file in this list
* Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')
log_files

['c:/Users/Admin/Documents/GitHub/SQLtest/data/log_data/2018/11/2018-11-01-events.json',
 'c:/Users/Admin/Documents/GitHub/SQLtest/data/log_data/2018/11/2018-11-02-events.json',
 'c:/Users/Admin/Documents/GitHub/SQLtest/data/log_data/2018/11/2018-11-03-events.json',
 'c:/Users/Admin/Documents/GitHub/SQLtest/data/log_data/2018/11/2018-11-04-events.json',
 'c:/Users/Admin/Documents/GitHub/SQLtest/data/log_data/2018/11/2018-11-05-events.json',
 'c:/Users/Admin/Documents/GitHub/SQLtest/data/log_data/2018/11/2018-11-06-events.json',
 'c:/Users/Admin/Documents/GitHub/SQLtest/data/log_data/2018/11/2018-11-07-events.json',
 'c:/Users/Admin/Documents/GitHub/SQLtest/data/log_data/2018/11/2018-11-08-events.json',
 'c:/Users/Admin/Documents/GitHub/SQLtest/data/log_data/2018/11/2018-11-09-events.json',
 'c:/Users/Admin/Documents/GitHub/SQLtest/data/log_data/2018/11/2018-11-10-events.json',
 'c:/Users/Admin/Documents/GitHub/SQLtest/data/log_data/2018/11/2018-11-11-events.json',
 'c:/Users/Admin/Docu

In [12]:
df_log = pd.read_json(log_files[0], lines = True)
df_log

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
8,Blue October / Imogen Heap,Logged In,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Congratulations,200,1541107493796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
9,Girl Talk,Logged In,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Once again,200,1541107734796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## time Table
### Extract Data for Time Table
* Filter records by NextSong action
* Convert the ts timestamp column to datetime
    * Hint: the current timestamp is in milliseconds
* Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts * column and set time_data to a list containing these values in order
    * Hint: use pandas' dt attribute to access easily datetimelike properties.
* Specify labels for these columns and set to column_labels
* Create a dataframe, time_df, containing the time data for this file by combining column_labels and time_data into a dictionary and converting this into a dataframe

In [13]:
df_log = df_log[df_log['page'] == 'NextSong']
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [14]:
ts = pd.to_datetime(df_log['ts'], unit = 'ms')
ts

2    2018-11-01 21:01:46.796
4    2018-11-01 21:05:52.796
5    2018-11-01 21:08:16.796
6    2018-11-01 21:11:13.796
7    2018-11-01 21:17:33.796
8    2018-11-01 21:24:53.796
9    2018-11-01 21:28:54.796
10   2018-11-01 21:42:00.796
12   2018-11-01 21:52:05.796
13   2018-11-01 21:55:25.796
14   2018-11-01 22:23:14.796
Name: ts, dtype: datetime64[ns]

In [15]:
data_time = []
for row in ts:
    data_time.append([row, row.hour, row.day, row.week, row.month, row.year, row.day_name()])
col_labels = ('start_time', 'hour', 'day', 'week', 'month','year','weekday')

In [16]:
df_time = pd.DataFrame.from_records(data_time, columns=col_labels)
df_time.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-01 21:01:46.796,21,1,44,11,2018,Thursday
1,2018-11-01 21:05:52.796,21,1,44,11,2018,Thursday
2,2018-11-01 21:08:16.796,21,1,44,11,2018,Thursday
3,2018-11-01 21:11:13.796,21,1,44,11,2018,Thursday
4,2018-11-01 21:17:33.796,21,1,44,11,2018,Thursday


In [17]:
for i, row in df_time.iterrows():
    cur.execute(sql_queries.insert_record.time, list(row))
    conn.commit()

## users Table
Extract Data for Users Table
* Select columns for user ID, first name, last name, gender and level and set to user_df

In [18]:
df_user = df_log[['userId', 'firstName', 'lastName', 'gender', 'level']]
df_user

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free
5,8,Kaylee,Summers,F,free
6,8,Kaylee,Summers,F,free
7,8,Kaylee,Summers,F,free
8,8,Kaylee,Summers,F,free
9,8,Kaylee,Summers,F,free
10,10,Sylvie,Cruz,F,free
12,26,Ryan,Smith,M,free
13,26,Ryan,Smith,M,free


In [19]:
for i, row in df_user.iterrows():
    cur.execute(sql_queries.insert_record.user, list(row))
    conn.commit()


## songplays Table
###Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the songplays table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.

* Implement the song_select query in sql_queries.py to find the song ID and artist ID based on the title, artist name, and duration of a song.
* Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to songplay_data
### Insert Records into Songplays Table
* Implement the songplay_table_insert query and run the cell below to insert records for the songplay actions in this log file into the songplays table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the songplays table in the sparkify database.

In [20]:
for i, row in df_log.iterrows():

    cur.execute(sql_queries.select.song, (row.song, row.artist, row.length))
    result = cur.fetchone()

    if result:
        songid, artistid = results
    else:
        songid, artistid = None, None

    data_songplay = (i, pd.to_datetime(row.ts, unit='ms'), int(row.userId), row.level, songid, artist_id, row.sessionId, row.location, row.userAgent)

    cur.execute(sql_queries.insert_record.songplay, data_songplay)
    conn.commit()

In [21]:
conn.close()